In [30]:
import tensorflow as tf
import pandas as pd
import datetime
import numpy as np
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from sklearn.model_selection import train_test_split

In [31]:
path = 'C:/Users/Flesch Family/Desktop/USF/Capstone-Project/'
tweets = pd.read_csv(path + 'tweet_global_warming.csv', encoding = 'unicode_escape')
tweets = tweets.dropna()
tweets = tweets[tweets['existence.confidence'] > 0.59]
tweets['binary'] = np.where(tweets['existence'].isin(['Yes', 'Y']), 1, 0)
tweets['binary'].value_counts()

1    2939
0     966
Name: binary, dtype: int64

In [32]:
pulled = pd.read_csv(path + 'pulled_data.csv', encoding = 'unicode_escape')
pulled['tweet'], pulled['creation_date'] = pulled['tweet'].astype(str), pulled['creation_date'].astype(str)
pulled = pulled.dropna()
pulled['creation_year'] = [datetime.datetime.strptime(_, '%Y-%m-%dT%H:%M:%S.%fz').strftime('%Y') for _ in pulled['creation_date']]
pulled['creation_month'] = [datetime.datetime.strptime(_, '%Y-%m-%dT%H:%M:%S.%fz').strftime('%m') for _ in pulled['creation_date']]
pulled['creation_y/m'] = [datetime.datetime.strptime(_, '%Y-%m-%dT%H:%M:%S.%fz').strftime('%Y-%m-%d') for _ in pulled['creation_date']]
pulled

c:\users\flesch family\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (0,3,4,5,8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,author_id,tweet,creation_date,retweets,likes,reply_count,name,screenname,user_id,location,verified,creation_year,creation_month,creation_y/m
0,12583392,Sci/Tech Technology key to fighting climate ch...,2008-04-02T23:29:17.000Z,0,0,0,Liveone,Liveone,12583392,india,False,2008,04,2008-04-02
1,13453152,How are more taxes going to solve global warmi...,2008-04-02T23:28:22.000Z,0,0,0,adamroark,adamroark,13453152,No Location,False,2008,04,2008-04-02
2,14218363,"Youtube ate my life - I saw 'more cowbell"", pr...",2008-04-02T22:12:52.000Z,0,0,0,JackieJack,JackieJack,14218363,San Francisco,False,2008,04,2008-04-02
3,9523892,Craig Venter on the dangers and opportunities ...,2008-04-02T21:56:47.000Z,0,0,0,Bill Day,billday,9523892,No Location,False,2008,04,2008-04-02
4,12715772,Across the Nation... States Suing EPA Over Glo...,2008-04-02T21:54:00.000Z,0,0,0,Rss News Feed,DCNewsFeed,12715772,Washington DC,False,2008,04,2008-04-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482107,1061495770807906304,Climate Change and Risk Reduction Management.,2021-03-29T23:48:21.000Z,0,0,1,Holy Mogen,xsxsxlxl,1061495770807906304,"City of Ilagan, Isabela",False,2021,03,2021-03-29
482108,1348099350509940737,Many experts believe that to fight climate cha...,2021-03-29T23:48:17.000Z,0,2,0,GLGY 297 @UofCalgary,GLGY297_Calgary,1348099350509940737,"Calgary, Alberta",False,2021,03,2021-03-29
482109,471205397,@JusticeMyanmar Ppl in Australia frequently pr...,2021-03-29T23:48:15.000Z,0,2,1,p2khaing,phoopwintkhaing,471205397,United States,False,2021,03,2021-03-29
482110,1327755170814410752,@charliespiering Isn't the wasteful use of gas...,2021-03-29T23:48:15.000Z,0,0,0,Mo Ashley,monique39966560,1327755170814410752,No Location,False,2021,03,2021-03-29


In [33]:
clean = pd.DataFrame({
    'DATA_COLUMN' : tweets['tweet'],
    'LABEL_COLUMN' : tweets['binary']
})
train, test = train_test_split(clean, test_size = 0.2)

In [6]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [35]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

c:\users\flesch family\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2149: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


196/196 [==============================] - 2662s 13s/step - loss: 0.4226 - accuracy: 0.8061 - val_loss: 0.3540 - val_accuracy: 0.8668
Epoch 2/2
196/196 [==============================] - 2452s 13s/step - loss: 0.1392 - accuracy: 0.9508 - val_loss: 0.4287 - val_accuracy: 0.8656

196/196 [==============================] - 3230s 16s/step - loss: 0.0520 - accuracy: 0.9844 - val_loss: 0.5270 - val_accuracy: 0.8720
Epoch 2/3
196/196 [==============================] - 2525s 13s/step - loss: 0.0315 - accuracy: 0.9874 - val_loss: 0.6623 - val_accuracy: 0.8720
Epoch 3/3
196/196 [==============================] - 2587s 13s/step - loss: 0.0192 - accuracy: 0.9936 - val_loss: 0.7404 - val_accuracy: 0.8694


In [36]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
    196/Unknown - 2972s 15s/step - loss: 0.4538 - accuracy: 0.7808WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config o

In [49]:
master = pd.DataFrame()
for year in pulled['creation_y/m'].unique():
    df_holder = pulled[pulled['creation_y/m'] == year]
    tf_batch = tokenizer(list(df_holder['tweet']), max_length=128, padding=True, truncation=True, return_tensors='tf')
    tf_outputs = model(tf_batch)
    tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
    labels = [0, 1]
    label = tf.argmax(tf_predictions, axis=1)
    label = label.numpy()
    df_holder['pred'] = label
    master = pd.concat([master, df_holder])
    print(year, end = '\r')

<ipython-input-49-58b210145ec6>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_holder['pred'] = label


In [50]:
master['pred'].value_counts()

1    407692
0     74275
Name: pred, dtype: int64

In [51]:
master.to_csv(path + 'pulled_pred.csv', index = False)
master

,author_id,tweet,creation_date,retweets,likes,reply_count,name,screenname,user_id,location,verified,creation_year,creation_month,creation_y/m,pred
0,12583392,Sci/Tech Technology key to fighting climate ch...,2008-04-02T23:29:17.000Z,0,0,0,Liveone,Liveone,12583392,india,False,2008,04,2008-04-02,1
1,13453152,How are more taxes going to solve global warmi...,2008-04-02T23:28:22.000Z,0,0,0,adamroark,adamroark,13453152,No Location,False,2008,04,2008-04-02,1
2,14218363,"Youtube ate my life - I saw 'more cowbell"", pr...",2008-04-02T22:12:52.000Z,0,0,0,JackieJack,JackieJack,14218363,San Francisco,False,2008,04,2008-04-02,0
3,9523892,Craig Venter on the dangers and opportunities ...,2008-04-02T21:56:47.000Z,0,0,0,Bill Day,billday,9523892,No Location,False,2008,04,2008-04-02,1
4,12715772,Across the Nation... States Suing EPA Over Glo...,2008-04-02T21:54:00.000Z,0,0,0,Rss News Feed,DCNewsFeed,12715772,Washington DC,False,2008,04,2008-04-02,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482107,1061495770807906304,Climate Change and Risk Reduction Management.,2021-03-29T23:48:21.000Z,0,0,1,Holy Mogen,xsxsxlxl,1061495770807906304,"City of Ilagan, Isabela",False,2021,03,2021-03-29,1
482108,1348099350509940737,Many experts believe that to fight climate cha...,2021-03-29T23:48:17.000Z,0,2,0,GLGY 297 @UofCalgary,GLGY297_Calgary,1348099350509940737,"Calgary, Alberta",False,2021,03,2021-03-29,1
482109,471205397,@JusticeMyanmar Ppl in Australia frequently pr...,2021-03-29T23:48:15.000Z,0,2,1,p2khaing,phoopwintkhaing,471205397,United States,False,2021,03,2021-03-29,0
482110,1327755170814410752,@charliespiering Isn't the wasteful use of gas...,2021-03-29T23:48:15.000Z,0,0,0,Mo Ashley,monique39966560,1327755170814410752,No Location,False,2021,03,2021-03-29,1


In [17]:
list(master[master['pred'] == 1]['tweet'])

['Â« If climate change is â\x80\x9ca greater threat than terrorism,â\x80\x9d should governments criminalize â\x80\x9cthe gloriï¬\x81cation of travelâ\x80\x9d Â»',
 '$2.5 mil for a "Waterfront Duck Pond Park? 4.8 mil for polar bears? $400 mil for "Global Warming Research"? Pitchforks and torches time?',
 'LOL Levin just said "Liberals oppose real pork (pigs, hogs) (because of fake global warming) but they love Washington pork."',
 'http://twitpic.com/1d0y9 - OKC Geological Society Discussion Group, 10 February 2009, entitled "Climate Change Facts: Should Oklahoma Be ...',
 'Global warming in effect. Im blasting the a.c.',
 'Snow?  Seriously?  What is wrong with this place & where is that "global warming" thing...',
 'Heute ist World Environment Day - Motto: "UNite to combat climate change! Der perfekte Tag, um unsere Kampagne offiziell zu starten!',
 "Why I heart Democrats :U.S. House Vote on Climate Change Bill - 211 Dems +  8 Republicans voted 'yes'; 44 Dems + 168 Republicans voted 'n

In [37]:
tf_batch = tokenizer(list(tweets['tweet']), max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = [0, 1]
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()

In [38]:
tweets['pred'] = label
tweets['pred'].value_counts()

1    3026
0     879
Name: pred, dtype: int64

In [39]:
test = pd.read_csv(path + 'test.csv')
tf_batch = tokenizer(list(test['processed']), max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = [0, 1]
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
test['pred'] = label
test['pred'].value_counts()

1    857
0    143
Name: pred, dtype: int64

In [43]:
test = pd.read_csv(path + 'test.csv')
tf_batch = tokenizer(list(test['tweet']), max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = [0, 1]
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
test['pred'] = label
test['pred'].value_counts()

1    851
0    149
Name: pred, dtype: int64